In [ ]:
import pandas as pd

In [ ]:
df_train=pd.read_csv('../input/cola-train-test/COLA_TRAIN.csv')

In [ ]:
df_test=pd.read_csv('../input/cola-train-test/COLA_TEST.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
###Drop Nan Values
df_train=df_train.dropna()
df_test=df_test.dropna()

In [ ]:
## Get the Independent Features

X=df_train.drop('Label',axis=1)
x=df_test.drop('Label',axis=1)

In [ ]:
## Get the Dependent features
Y=df_train['Label']
y=df_test['Label']

In [ ]:
Y.value_counts()

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=15000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
test_message=x.copy()

In [ ]:
messages['text'][1]

In [ ]:
messages.reset_index(inplace=True)
test_message.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
corpus_test=[]
for i in range(0, len(messages)):

    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    
    
for j in range(0, len(test_message)):

    review = re.sub('[^a-zA-Z]', ' ', test_message['text'][j])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_test.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr_test=[one_hot(words,voc_size)for words in corpus_test] 


### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_length)

In [ ]:
embedded_docs[0]

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
## Creating model
embedding_vector_features=60
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

model1.add(Bidirectional(LSTM(1000)))
model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_train=np.array(embedded_docs)
y_train=np.array(Y)

X_test=np.array(embedded_docs_test)
y_test=np.array(y)

### Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=32,batch_size=64)

### Performance Metrics And Accuracy

In [ ]:

y_pred1=model1.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))